In [ ]:
# Import Libraries
import warnings
warnings.filterwarnings('ignore')
import time
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [ ]:
#모델 라이브러리 가져오기 
import itertools
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten,Dense,BatchNormalization,Activation,Dropout
import tensorflow as tf
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
#import resnet
from tensorflow.keras.applications import resnet50

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
x_train = np.load('/content/drive/Shareddrives/2022-1 KUBIG 음악추천시스템/melspectogram.npy')

In [ ]:
target = []
for i in range(200):
  target.append(0)
for i in range(200):
  target.append(1)
for i in range(200):
  target.append(2)
for i in range(200):
  target.append(3)
for i in range(200):
  target.append(4)


In [ ]:
y_train = np.array(target)

In [ ]:
import cv2
cv2.resize(x_train[0], (256, 256), interpolation = cv2.INTER_LANCZOS4).shape
new = []
for i in range(1000):
  new.append(cv2.resize(x_train[i], (256, 256), interpolation = cv2.INTER_LANCZOS4))

In [ ]:
new2 = []
for i in range(1000):
  new2.append(np.array([new[i], new[i], new[i]]))

In [ ]:
x_train = np.array(new2)

In [ ]:
x_train.shape

(1000, 3, 256, 256)

In [ ]:
x_train = np.swapaxes(np.swapaxes(x_train, 1, 2), 2, 3)

In [ ]:
x_train.shape

(1000, 256, 256, 3)

In [ ]:
# train / val / test

# 7 : 2 : 1



In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, random_state = 256, test_size = 0.3)

In [ ]:
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, random_state = 256, test_size = 0.3)

In [ ]:
from tensorflow.keras.applications import MobileNetV3Small

In [ ]:
model=resnet50.ResNet50(include_top = False, weights = 'imagenet', input_shape = (256, 256, 3))
flattened = tf.keras.layers.Flatten()(model.output)

fc1 = tf.keras.layers.Dense(5, activation='softmax', name="AddedDense2")(flattened)

model = tf.keras.models.Model(inputs=model.input, outputs=fc1) 

In [ ]:
#모델 컴파일
model.compile(optimizer ='adamax',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

print('Model Details are : ')
print(model.summary())

Model Details are : 
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 262, 262, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 128, 128, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                         

In [ ]:
#model fit
from keras.callbacks import ModelCheckpoint, EarlyStopping   
callbacks_list = [  
    ModelCheckpoint('/content/drive/MyDrive/musicrecommendation_resent.h5', monitor='val_accuracy', mode='max', verbose = 1, save_best_only = True),
    EarlyStopping(patience=8, verbose = 1),
    ReduceLROnPlateau(patience=4, verbose=1)]  

ThisModel = model.fit(X_train, y_train, validation_data = (X_val, y_val), batch_size=32, epochs=100, callbacks=callbacks_list, verbose=1)

Epoch 1/100
22/22 [==============================] - ETA: 0s - loss: 4.3231 - accuracy: 0.4800
Epoch 1: val_accuracy improved from -inf to 0.18571, saving model to /content/drive/MyDrive/musicrecommendation_resent.h5
22/22 [==============================] - 42s 1s/step - loss: 4.3231 - accuracy: 0.4800 - val_loss: 27.5034 - val_accuracy: 0.1857 - lr: 0.0010
Epoch 2/100
22/22 [==============================] - ETA: 0s - loss: 2.5528 - accuracy: 0.6900
Epoch 2: val_accuracy improved from 0.18571 to 0.19524, saving model to /content/drive/MyDrive/musicrecommendation_resent.h5
22/22 [==============================] - 25s 1s/step - loss: 2.5528 - accuracy: 0.6900 - val_loss: 268.3409 - val_accuracy: 0.1952 - lr: 0.0010
Epoch 3/100
22/22 [==============================] - ETA: 0s - loss: 1.1335 - accuracy: 0.8071
Epoch 3: val_accuracy improved from 0.19524 to 0.40476, saving model to /content/drive/MyDrive/musicrecommendation_resent.h5
22/22 [==============================] - 24s 1s/step - l

In [ ]:
pred = np.argmax(model.predict(X_test), axis = 1)

In [ ]:
y_test

array([0, 3, 0, 0, 3, 4, 4, 4, 3, 0, 3, 0, 4, 1, 0, 2, 1, 2, 1, 3, 2, 1,
       4, 4, 0, 2, 3, 3, 1, 3, 4, 2, 1, 1, 1, 4, 1, 0, 1, 0, 3, 3, 0, 2,
       1, 3, 4, 2, 4, 1, 3, 0, 2, 0, 2, 1, 0, 4, 4, 1, 4, 3, 1, 1, 1, 1,
       1, 4, 0, 2, 0, 2, 4, 1, 3, 1, 1, 1, 4, 1, 2, 3, 4, 1, 1, 3, 4, 3,
       3, 2])

In [ ]:
sum = 0
for i in range(len(pred)):
  if pred[i] == y_test[i]:
    sum = sum + 1
sum / len(pred) * 100

In [ ]:
sum / len(pred) * 100

65.55555555555556

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.62      0.67      0.65        15
           1       0.78      0.81      0.79        26
           2       0.71      0.38      0.50        13
           3       0.92      0.61      0.73        18
           4       0.43      0.67      0.52        18

    accuracy                           0.66        90
   macro avg       0.69      0.63      0.64        90
weighted avg       0.70      0.66      0.66        90



In [ ]:
## simplernn
## lstm
## gru

## resnet50
## inception / xception
## senet (그냥...)
## vision transformer

## cosine similiarity
## p-norm


## 1. 본인 장르 30개 더
## 2. 7788 / 상준 승은 지호 지우
## 3. 지호 지우 추천시스템 (함수 만들기) - cv 모델 느낌만
## 4. 상준 승은 다 하기